# Crime and public infrastructure in coastal US cities
By Leonardo Nicoletti and Caroline Cullinan

### To what extent do elements of the urban fabric influence crime rates in major US cities?
Internationally, crime devastates countries. Increased crime results in harmful effects for legal economies. Investments from foreign and domestic sources are discouraged, and firms become less competitive as a result of crime. Crime also threatens the personal security of a country’s citizens, thus negatively impacting the overall health and well-being of individuals. Those victimized by crime experience mental-health disorders such as insomnia, depression, and anxiety at increased rates. Therefore, the negative health effects associated with crime can contribute to social disengagement, thus weakening overall social cohesion of communities. 

In order to address perpetuators of crime, Wilson & Kelling proposed the “broken window theory” (1982). Such theory disputes that regardless of neighborhood, neglected property encourages further neglect and destruction of property. Such theory insists that increased incidents of crime are associated with increased instances of community neglect. Further argued, the “broken window theory” suggests that minor forms of neglect, such as garbage-littered streets and broken street lamps, have the greatest contributions to criminal activity, for the existence of disorder may allude that the actions of individuals’ go unaccounted for. 

To test the relevance of this theory in current times, the relationship between crime and elements of the urban fabric that contribute to the “cleanliness” and “care” of streets are assessed across five United States (US) cities; San Francisco, Los Angeles, Boston, Philadelphia, and Washington D.C.. Such analysis examines if trash can availability, street light presence, and the existence of trees in urban communities relate to crime.  With the use of a predictive model, such analysis also examines if the existence of such elements (trash cans, street lights, and trees) can predict instances of crime.




## Part 1: Data Preprocessing

In [ ]:
#let's start by setting our directory and importing relevant packages
setwd("C:/Users/Leonardo/Documents/TU_Delft/EPA_Year1/EPA1315/FinalProject")
#install required For this project
install.packages(c('wesanderson', 'RColorBrewer', 'sf', 'ggmap', 'maps', 'mapdata', 
                   'rgeos', 'tidyverse', 'lwgeom', 'stringr', 'plotly', 'fastDummies')
#Libraries required For this project
library(fastDummies)
library(rgdal)
library(ggplot2)
library(wesanderson)
library(RColorBrewer)
library(sf)
library(ggplot2)
library(ggmap)
library(maps)
library(mapdata)
library(rgeos)
library(tidyverse)
library(lwgeom)
library(stringr)
library(dplyr)

### 1.1 San Francisco

In [ ]:
#now let's import data for san francisco
#Open SF public infrastructure assets
SF_assets <- st_read("Data/SF/Assets/geo_export_d7f59e21-8bca-42e3-b982-a40615a1dba7.shp")
#open SF trees
SF_trees <- st_read("Data/SF/Trees/geo_export_23873827-8da7-4487-9fe7-bf9ce37d7c7d.shp")
#open SF precincts
SF_prcnts <- st_read("Data/SF/Precincts/geo_export_0665516f-f29b-4978-8f0a-b270abce6a43.shp")
#open SF crime
SF_crime <- st_read("Data/SF/Crime/geo_export_63c82a7d-2120-4694-9aab-8c844c26bd1e.shp")

In [ ]:
#reproject spatial layers
SF_prcnts <- st_transform(SF_prcnts, "+proj=longlat +datum=WGS84 +no_defs")
SF_assets <- st_transform(SF_assets, "+proj=longlat +datum=WGS84 +no_defs")
SF_trcts <- st_transform(SF_trcts, "+proj=longlat +datum=WGS84 +no_defs")
SF_crime <- st_transform(SF_crime, "+proj=longlat +datum=WGS84 +no_defs")
SF_trees <- st_transform(SF_trees, "+proj=longlat +datum=WGS84 +no_defs")
#check that all spatial layers are in the same coordinate system
st_crs(SF_assets)
st_crs(SF_trcts)
st_crs(SF_crime)
st_crs(SF_trees)
st_crs(SF_prcnts)

In [78]:
#SF_assets has information about many types of public assets (trash cans, benches, light posts etc.)
#levels(SF_assets$asset_type)
#let's extract the public assets of interest for our analysis ("Trash Can" and "Lamp Post")
#SF_trashcans
SF_trashcans <- SF_assets %>%
  dplyr::select(asset_type, geometry) %>%   
  filter(str_detect(asset_type, c('Trash Can')))
SF_trashcans$trashcount <- 1

#SF_streetlights
SF_streetlights <- SF_assets %>%
  dplyr::select(asset_type, geometry) %>%   
  filter(str_detect(asset_type, c('Lamp Post')))
SF_streetlights$lightcount <- 1

#now let's clean up our other layers to include only columns of interest
#precincts
SF_prcnts <- select(SF_prcnts, 'neighrep', 'prec_2012', 'geometry')
#trees
SF_trees <- subset(SF_trees, is.na(SF_trees$xcoord) == FALSE) 
SF_trees <- select(SF_trees, 'treeid', 'xcoord', 'ycoord', 'geometry')
SF_trees$treecount <- 1
#crime
SF_crime <- subset(SF_crime, is.na(SF_crime$supervisor) == FALSE)
SF_crime <- select(SF_crime, 'analysis_n','geometry')
SF_crime$crimecount <- 1

In [79]:
#let's write a function that takes in all the variables, performs spatial joins on points inside a given geometry (precincts), and summarizes 
#by geometry

SF_trct_varsfunc <- function(geom, var1, var2, var3, var4) {
    #var1=trees; var2=crime; var3=streetlights; var4=trashcans
    #join first variable
    SF_trcts_var1 <- st_join(geom, left = TRUE, var1[length(var1)])
    SF_trcts_var1 <- group_by(SF_trcts_var1, neighrep, prec_2012)
    SF_trcts_var1 <- summarise(SF_trcts_var1, treecount = sum(treecount))
    #join second variable
    SF_trcts_var12 <- st_join(SF_trcts_var1, left = TRUE, var2[length(var2)])
    SF_trcts_var12 <- group_by(SF_trcts_var12, neighrep, prec_2012, treecount)
    SF_trcts_var12 <- summarise(SF_trcts_var12, crimecount = sum(crimecount))
    #join third variable
    SF_trcts_var123 <- st_join(SF_trcts_var12, left = TRUE, var3[length(var3)])
    SF_trcts_var123 <- group_by(SF_trcts_var123, neighrep, prec_2012, treecount, crimecount)
    SF_trcts_var123 <- summarise(SF_trcts_var123, lightcount = sum(lightcount))
    #join fourth variable
    SF_trcts_var1234 <- st_join(SF_trcts_var123, left = TRUE, var4[length(var4)])
    SF_trcts_var1234 <- group_by(SF_trcts_var1234, neighrep, prec_2012, treecount, crimecount, lightcount)
    SF_trcts_var1234 <- summarise(SF_trcts_var1234, trashcount = sum(trashcount))

    #replace NA values with 0 (meaning that the count for a given var is 0 if there was no point within a polygon)
    SF_trcts_var1234[is.na(SF_trcts_var1234)] <- 0
    SF_trcts_vars <- SF_trcts_var1234
   return(SF_trcts_vars)
}

In [ ]:
#running our function for san francisco
SF_prcnts_vars <- SF_trct_varsfunc(SF_prcnts, SF_trees, SF_crime, SF_streetlights, SF_trashcans)
#save file
st_write(SF_prcnts_vars, "Data/SF/ProcessedData/SF_prcnts_vars.shp")

#### Minimizing error and normalizing our data

Because census tracts all differ in sizes, it's easy to misrepresent the intensity of our variables. In order to avoid this, we first need to normalize our variables by area. For each variable, we divide the row values by the tract area.

Next, we also need to normalize our variables so that they can be compared among eachother. For this we will use R's standardize function, which will put all variables on the same scale.

In [ ]:
#first we calculate the area of each tract in km2
SF_prcnts_vars$km2 <- as.numeric((st_area(SF_prcnts_vars)/1000000))
#next we divide each variable by the tract area
SF_prcnts_vars$crimecount <- SF_prcnts_vars$crimecount/SF_prcnts_vars$km2
SF_prcnts_vars$lightcount <- SF_prcnts_vars$lightcount/SF_prcnts_vars$km2
SF_prcnts_vars$trashcount <- SF_prcnts_vars$trashcount/SF_prcnts_vars$km2
SF_prcnts_vars$treecount <- SF_prcnts_vars$treecount/SF_prcnts_vars$km2
#reorder dataframe
SF_prcnts_vars <- SF_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'neighrep', 'prec_2012', 'geometry', 'km2')]
#save file
st_write(SF_prcnts_vars, "Data/SF/ProcessedData/SF_prcnts_vars_norm.shp")

### 1.2 Los Angeles

In [27]:
#now let's import data for Los Angeles
#Open LA public streetlights
LA_streetlights <- st_read("Data/LA/StreetLights/geo_export_179d7440-79b6-4137-b1d8-8ff5286a9b47.shp")
#open LA trashcans
LA_trashcans <- st_read("Data/LA/TrashCans/LA_City_Receptacles.shp")
#open LA trees
LA_trees <- st_read("Data/LA/Trees/output.shp")
#open LA precincts
LA_prcnts <- st_read("Data/LA/Precincts/geo_export_bc1bd767-83ec-499a-98e7-5fdacb39b78c.shp")
#open LA census tracts
LA_trcts <- st_read("Data/LA/CensusTracts/Neighborhood_Council_Boundaries_2018.shp")
#open LA crime
LA_crime <- read_csv("Data/LA/Crime/Crime_Data_from_2010_to_Present.csv")

Reading layer `geo_export_179d7440-79b6-4137-b1d8-8ff5286a9b47' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\LA\StreetLights\geo_export_179d7440-79b6-4137-b1d8-8ff5286a9b47.shp' using driver `ESRI Shapefile'
Simple feature collection with 216694 features and 11 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -118.6681 ymin: 33.70677 xmax: -118.1551 ymax: 34.33452
epsg (SRID):    4326
proj4string:    +proj=longlat +ellps=WGS84 +no_defs
Reading layer `LA_City_Receptacles' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\LA\TrashCans\LA_City_Receptacles.shp' using driver `ESRI Shapefile'
Simple feature collection with 9713 features and 10 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -118.657 ymin: 33.70693 xmax: -118.1555 ymax: 34.32207
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `output' from data source `C:\User

In [25]:
#subset LA_crime for 2018 and onwards
LA_crime <- LA_crime %>% separate('DATE OCC', into = paste0('DATE OCC', 1:4), sep = '[/ ]')
LA_crime <- subset(LA_crime, LA_crime['DATE OCC3'] == '2018' | LA_crime['DATE OCC3'] == '2019')

#convert crime csv to geospatial object
LA_crime <- st_as_sf(LA_crime, coords = c("LON", "LAT"), 
                 crs = 4326, agr = "constant")

st_write(LA_crime,'Data/LA/Crime/Crime_Data_from_2018_to_Present.shp')

Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"

Writing layer `Crime_Data_from_2018_to_Present' to data source `Data/LA/Crime/Crime_Data_from_2018_to_Present.shp' using driver `ESRI Shapefile'
Writing 402802 features with 29 fields and geometry type Point.


In [85]:
#reproject streetlights and precincts to common coordinate system
LA_streetlights <- st_transform(LA_streetlights, "+proj=longlat +datum=WGS84 +no_defs")
LA_prcnts <- st_transform(LA_prcnts, "+proj=longlat +datum=WGS84 +no_defs")
#check that all spatial layers are in the same coordinate system
st_crs(LA_streetlights)
st_crs(LA_trashcans)
st_crs(LA_trees)
st_crs(LA_trcts)
st_crs(LA_prcnts)
st_crs(LA_crime)

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

In [86]:
#now let's clean up our layers to include only columns of interest
#tracts layer
LA_trcts <- select(LA_trcts, 'Name', 'geometry')
LA_prcnts <- select(LA_prcnts, 'cnsld', 'geometry')
#trashcans
LA_trashcans <- select(LA_trashcans, 'LAPDGrid', 'geometry')
LA_trashcans$trashcount <- 1
#streetlights
LA_streetlights <- select(LA_streetlights, 'stlid', 'geometry')
LA_streetlights$lightcount <- 1
#trees
LA_trees <- select(LA_trees, 'TreeID', 'geometry')
LA_trees$treecount <- 1
#crime
LA_crime <- select(LA_crime, 'Date Rptd','geometry')
LA_crime$crimecount <- 1

In [87]:
#let's write a function that takes in all the variables, perform spatial joins on points inside census tracts, and summarizes by census tract 
LA_prcnts_varsfunc <- function(geom, var1, var2, var3, var4) {
    #var1=trees; var2=crime; var3=streetlights; var4=trashcans
    #join first variable
    SF_trcts_var1 <- st_join(geom, left = TRUE, var1[length(var1)])
    SF_trcts_var1 <- group_by(SF_trcts_var1, cnsld)
    SF_trcts_var1 <- summarise(SF_trcts_var1, treecount = sum(treecount))
    #join second variable
    SF_trcts_var12 <- st_join(SF_trcts_var1, left = TRUE, var2[length(var2)])
    SF_trcts_var12 <- group_by(SF_trcts_var12, cnsld, treecount)
    SF_trcts_var12 <- summarise(SF_trcts_var12, crimecount = sum(crimecount))
    #join third variable
    SF_trcts_var123 <- st_join(SF_trcts_var12, left = TRUE, var3[length(var3)])
    SF_trcts_var123 <- group_by(SF_trcts_var123, cnsld, treecount, crimecount)
    SF_trcts_var123 <- summarise(SF_trcts_var123, lightcount = sum(lightcount))
    #join fourth variable
    SF_trcts_var1234 <- st_join(SF_trcts_var123, left = TRUE, var4[length(var4)])
    SF_trcts_var1234 <- group_by(SF_trcts_var1234, cnsld, treecount, crimecount, lightcount)
    SF_trcts_var1234 <- summarise(SF_trcts_var1234, trashcount = sum(trashcount))

    #replace NA values with 0 (meaning that the count for a given var is 0 if there was no point within a polygon)
    SF_trcts_var1234[is.na(SF_trcts_var1234)] <- 0
    SF_trcts_vars <- SF_trcts_var1234
   return(SF_trcts_vars)
}

In [88]:
#running our spatial join function for Los Angeles
LA_prcnts_vars <- LA_prcnts_varsfunc(LA_prcnts, LA_trees, LA_crime, LA_streetlights, LA_trashcans)
#save file
st_write(LA_prcnts_vars, "Data/LA/ProcessedData/LA_prcnts_vars.shp")

although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar


Writing layer `LA_prcnts_vars' to data source `Data/LA/ProcessedData/LA_prcnts_vars.shp' using driver `ESRI Shapefile'
Writing 1627 features with 5 fields and geometry type Multi Polygon.


#### normalizing our data by area

In [89]:
#first we calculate the area of each tract in km2
LA_prcnts_vars$km2 <- as.numeric((st_area(LA_prcnts_vars)/1000000))
#next we divide each variable by the tract area
LA_prcnts_vars$crimecount <- LA_prcnts_vars$crimecount/LA_prcnts_vars$km2
LA_prcnts_vars$lightcount <- LA_prcnts_vars$lightcount/LA_prcnts_vars$km2
LA_prcnts_vars$trashcount <- LA_prcnts_vars$trashcount/LA_prcnts_vars$km2
LA_prcnts_vars$treecount <- LA_prcnts_vars$treecount/LA_prcnts_vars$km2
#reorder dataframe
LA_prcnts_vars <- LA_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'cnsld', 'geometry', 'km2')]
#save file
st_write(LA_prcnts_vars, "Data/LA/ProcessedData/LA_prcnts_vars_norm.shp")

Writing layer `LA_prcnts_vars_norm' to data source `Data/LA/ProcessedData/LA_prcnts_vars_norm.shp' using driver `ESRI Shapefile'
Writing 1627 features with 6 fields and geometry type Multi Polygon.


### 1.2 Philadelphia

In [2]:
#now let's import data for Los Angeles
#Open Philadelphia public streetlights
PH_streetlights <- st_read("Data/Philadelphia/StreetLights/Street_Poles.shp")
#open Philadelphia trashcans - Big Belly
PH_trashcans_bb <- st_read("Data/Philadelphia/TrashCans/wastebaskets_big_belly.shp")
#open Philadelphia trashcans - Wire
PH_trashcans_w <- st_read("Data/Philadelphia/TrashCans/WasteBaskets_Wire.shp")
#open Philadelphia trees
PH_trees <- st_read("Data/Philadelphia/Trees/PPR_StreetTrees.shp")
#open Philadelphia precincts
PH_prcnts <- st_read("Data/Philadelphia/Precincts/Political_Divisions.shp")
#open Philadelphia crime
PH_crime <- st_read("Data/Philadelphia/Crime/incidents_part1_part2.shp")

Reading layer `Street_Poles' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\Philadelphia\StreetLights\Street_Poles.shp' using driver `ESRI Shapefile'
Simple feature collection with 194749 features and 13 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -75.28108 ymin: 39.87517 xmax: -74.95891 ymax: 40.13786
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `wastebaskets_big_belly' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\Philadelphia\TrashCans\wastebaskets_big_belly.shp' using driver `ESRI Shapefile'
Simple feature collection with 968 features and 4 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -75.27019 ymin: 39.91665 xmax: -75.03349 ymax: 40.07439
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `WasteBaskets_Wire' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_

In [8]:
#subset LA_crime for 2018 and onwards
PH_crime <-  PH_crime[PH_crime$dispatch_d > as.Date("2018-01-01"),]

st_write(PH_crime,'Data/Philadelphia/Crime/Crime_Data_from_2018_to_Present.shp')

Writing layer `Crime_Data_from_2018_to_Present' to data source `Data/Philadelphia/Crime/Crime_Data_from_2018_to_Present.shp' using driver `ESRI Shapefile'
Writing 305220 features with 13 fields and geometry type Point.


In [10]:
#check that all spatial layers are in the same coordinate system
st_crs(PH_prcnts)
st_crs(PH_crime)
st_crs(PH_trees)
st_crs(PH_trashcans_bb)
st_crs(PH_trashcans_w)
st_crs(PH_streetlights)

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

Coordinate Reference System:
  EPSG: 4326 
  proj4string: "+proj=longlat +datum=WGS84 +no_defs"

In [33]:
#now let's clean up our layers to include only columns of interest
#precincts layer
PH_prcnts <- select(PH_prcnts, 'DIVISION_N', 'geometry')
#merge big belly and wire trashcans
PH_trashcans <- rbind(select(PH_trashcans_bb, 'geometry'), select(PH_trashcans_w, 'geometry'))
PH_trashcans$trashcount <- 1
#streetlights
PH_streetlights <- select(PH_streetlights, 'POLE_NUM', 'geometry')
PH_streetlights$lightcount <- 1
#trees
PH_trees <- select(PH_trees, 'OBJECTID', 'geometry')
PH_trees$treecount <- 1
#crime
PH_crime <- select(PH_crime, 'dispatch_d','geometry')
PH_crime$crimecount <- 1

In [35]:
#let's write a function that takes in all the variables, perform spatial joins on points inside census tracts, and summarizes by census tract 
PH_prcnts_varsfunc <- function(geom, var1, var2, var3, var4) {
    #var1=trees; var2=crime; var3=streetlights; var4=trashcans
    #join first variable
    SF_trcts_var1 <- st_join(geom, left = TRUE, var1[length(var1)])
    SF_trcts_var1 <- group_by(SF_trcts_var1, DIVISION_N)
    SF_trcts_var1 <- summarise(SF_trcts_var1, treecount = sum(treecount))
    #join second variable
    SF_trcts_var12 <- st_join(SF_trcts_var1, left = TRUE, var2[length(var2)])
    SF_trcts_var12 <- group_by(SF_trcts_var12, DIVISION_N, treecount)
    SF_trcts_var12 <- summarise(SF_trcts_var12, crimecount = sum(crimecount))
    #join third variable
    SF_trcts_var123 <- st_join(SF_trcts_var12, left = TRUE, var3[length(var3)])
    SF_trcts_var123 <- group_by(SF_trcts_var123, DIVISION_N, treecount, crimecount)
    SF_trcts_var123 <- summarise(SF_trcts_var123, lightcount = sum(lightcount))
    #join fourth variable
    SF_trcts_var1234 <- st_join(SF_trcts_var123, left = TRUE, var4[length(var4)])
    SF_trcts_var1234 <- group_by(SF_trcts_var1234, DIVISION_N, treecount, crimecount, lightcount)
    SF_trcts_var1234 <- summarise(SF_trcts_var1234, trashcount = sum(trashcount))

    #replace NA values with 0 (meaning that the count for a given var is 0 if there was no point within a polygon)
    SF_trcts_var1234[is.na(SF_trcts_var1234)] <- 0
    SF_trcts_vars <- SF_trcts_var1234
   return(SF_trcts_vars)
}

In [36]:
#running our spatial join function for Los Angeles
PH_prcnts_vars <- PH_prcnts_varsfunc(PH_prcnts, PH_trees, PH_crime, PH_streetlights, PH_trashcans)
#save file
st_write(PH_prcnts_vars, "Data/Philadelphia/ProcessedData/PH_prcnts_vars.shp")

although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar
although coordinates are longitude/latitude, st_intersects assumes that they are planar


Writing layer `PH_prcnts_vars' to data source `Data/Philadelphia/ProcessedData/PH_prcnts_vars.shp' using driver `ESRI Shapefile'
Writing 1703 features with 5 fields and geometry type Polygon.


In [38]:
#first we calculate the area of each tract in km2
PH_prcnts_vars$km2 <- as.numeric((st_area(PH_prcnts_vars)/1000000))
#next we divide each variable by the tract area
PH_prcnts_vars$crimecount <- PH_prcnts_vars$crimecount/PH_prcnts_vars$km2
PH_prcnts_vars$lightcount <- PH_prcnts_vars$lightcount/PH_prcnts_vars$km2
PH_prcnts_vars$trashcount <- PH_prcnts_vars$trashcount/PH_prcnts_vars$km2
PH_prcnts_vars$treecount <- PH_prcnts_vars$treecount/PH_prcnts_vars$km2
#reorder dataframe
PH_prcnts_vars <- PH_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'DIVISION_N', 'geometry', 'km2')]
#save file
st_write(PH_prcnts_vars, "Data/Philadelphia/ProcessedData/PH_prcnts_vars_norm.shp")

Writing layer `PH_prcnts_vars_norm' to data source `Data/Philadelphia/ProcessedData/PH_prcnts_vars_norm.shp' using driver `ESRI Shapefile'
Writing 1703 features with 6 fields and geometry type Polygon.


### 1.4 Washington DC

In [446]:
#now let's import data for Los Angeles
#Open DC public streetlights
DC_streetlights <- st_read("Data/DC/StreetLights/Street_Lights.shp")
#open DC trashcans
DC_trashcans <- st_read("Data/DC/TrashCans/Litter_Cans.shp")
#open DC trees
DC_trees <- st_read("Data/DC/Trees/Urban_Forestry_Street_Trees.shp")
#open DC census blocks
DC_blocks <- st_read("Data/DC/CensusBlocks/Census_Blocks__2010.shp")
#open DC precincts
DC_prcnts <- st_read("Data/DC/Precincts/Voting_Precinct__2012.shp")
#open DC crime
DC_crime_2018 <- st_read("Data/DC/Crime/Crime_Incidents_in_2018.shp")
DC_crime_2019 <- st_read("Data/DC/Crime/Crime_Incidents_in_2019.shp")

Reading layer `Street_Lights' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\DC\StreetLights\Street_Lights.shp' using driver `ESRI Shapefile'
Simple feature collection with 70829 features and 50 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -77.11656 ymin: 38.80571 xmax: -76.90958 ymax: 38.99534
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `Litter_Cans' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\DC\TrashCans\Litter_Cans.shp' using driver `ESRI Shapefile'
Simple feature collection with 6703 features and 22 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -77.11191 ymin: 38.81882 xmax: -76.91069 ymax: 38.9916
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `Urban_Forestry_Street_Trees' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Dat

In [449]:
#now let's clean up our layers to include only columns of interest
#census blocks layer
DC_blocks <- select(DC_blocks, 'GEOID', 'geometry')
#precincts layer
DC_prcnts <- select(DC_prcnts, 'NAME', 'geometry')
#merge big belly and wire trashcans
DC_trashcans <- select(DC_trashcans, 'LITTERID', 'geometry')
DC_trashcans$trashcount <- 1
#streetlights
DC_streetlights <- select(DC_streetlights, 'WARD', 'geometry')
DC_streetlights$lightcount <- 1
#trees
DC_trees <- select(DC_trees, 'WARD', 'geometry')
DC_trees$treecount <- 1
#crime
#merge DC_crime_2018 and 2019
DC_crime <-  rbind(select(DC_crime_2018, 'WARD', 'geometry'), select(DC_crime_2018, 'WARD', 'geometry'))
DC_crime$crimecount <- 1

In [451]:
#let's write a function that takes in all the variables, perform spatial joins on points inside census tracts, and summarizes by census tract 
DC_prcnts_varsfunc <- function(geom, var1, var2, var3, var4) {
    #var1=trees; var2=crime; var3=streetlights; var4=trashcans
    #join first variable
    SF_trcts_var1 <- st_join(geom, left = TRUE, var1[length(var1)])
    SF_trcts_var1 <- group_by(SF_trcts_var1, NAME)
    SF_trcts_var1 <- summarise(SF_trcts_var1, treecount = sum(treecount))
    #join second variable
    SF_trcts_var12 <- st_join(SF_trcts_var1, left = TRUE, var2[length(var2)])
    SF_trcts_var12 <- group_by(SF_trcts_var12, NAME, treecount)
    SF_trcts_var12 <- summarise(SF_trcts_var12, crimecount = sum(crimecount))
    #join third variable
    SF_trcts_var123 <- st_join(SF_trcts_var12, left = TRUE, var3[length(var3)])
    SF_trcts_var123 <- group_by(SF_trcts_var123, NAME, treecount, crimecount)
    SF_trcts_var123 <- summarise(SF_trcts_var123, lightcount = sum(lightcount))
    #join fourth variable
    SF_trcts_var1234 <- st_join(SF_trcts_var123, left = TRUE, var4[length(var4)])
    SF_trcts_var1234 <- group_by(SF_trcts_var1234, NAME, treecount, crimecount, lightcount)
    SF_trcts_var1234 <- summarise(SF_trcts_var1234, trashcount = sum(trashcount))

    #replace NA values with 0 (meaning that the count for a given var is 0 if there was no point within a polygon)
    SF_trcts_var1234[is.na(SF_trcts_var1234)] <- 0
    SF_trcts_vars <- SF_trcts_var1234
   return(SF_trcts_vars)
}

In [ ]:
#running our spatial join function for Los Angeles
DC_prcnts_vars <- DC_prcnts_varsfunc(DC_prcnts, DC_trees, DC_crime, DC_streetlights, DC_trashcans)
#save file
st_write(DC_prcnts_vars, "Data/DC/ProcessedData/DC_prcnts_vars.shp")

In [ ]:
#normalizing by population
#next we divide each variable by population
#DC_prcnts_vars$crimecount <- DC_prcnts_vars$crimecount/DC_prcnts_vars$P0010001
#DC_prcnts_vars$lightcount <- DC_prcnts_vars$lightcount/DC_prcnts_vars$P0010001
#DC_prcnts_vars$trashcount <- DC_prcnts_vars$trashcount/DC_prcnts_vars$P0010001
#DC_prcnts_vars$treecount <- DC_prcnts_vars$treecount/DC_prcnts_vars$P0010001
#reorder dataframe
#DC_prcnts_vars <- DC_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'geometry', 'P0010001')]
#save file
#st_write(DC_prcnts_vars, "Data/DC/ProcessedData/DC_blocks_vars_popnorm.shp")

In [272]:
#first we calculate the area of each precinct in km2
DC_prcnts_vars$km2 <- as.numeric((st_area(DC_prcnts_vars)/1000000))
#next we divide each variable by the tract area
DC_prcnts_vars$crimecount <- DC_prcnts_vars$crimecount/DC_prcnts_vars$km2
DC_prcnts_vars$lightcount <- DC_prcnts_vars$lightcount/DC_prcnts_vars$km2
DC_prcnts_vars$trashcount <- DC_prcnts_vars$trashcount/DC_prcnts_vars$km2
DC_prcnts_vars$treecount <- DC_prcnts_vars$treecount/DC_prcnts_vars$km2
#reorder dataframe
DC_prcnts_vars <- DC_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'geometry', 'km2')]
#save file
st_write(DC_prcnts_vars, "Data/DC/ProcessedData/DC_prcnts_vars_norm.shp")

### 1.5 Boston

In [ ]:
#Now let's import data for Boston

#Open Boston public streetlights
BO_streetlights <- st_read("Data/Boston/StreetLights/geo_export_6d166c39-621f-4ca5-8fa9-6de277c48821.shp")
#Open Boston trashcans
BO_trashcans <- read.csv("Data/Boston/TrashCans/big-belly-locations.csv")
#Open Boston trees
BO_trees <- st_read("Data/Boston/Trees/Trees.shp")
#Open Boston precincts
BO_prcnts <- st_read("Data/Boston/Precincts/Precincts_2017.shp")
#Open Boston crime
BO_crime <- read.csv("Data/Boston/Crime/tmp2a1yu35_.csv")

In [ ]:
#Convert csv files to shapefile (with minor clean)

#Crime
#Subset BO_crime for 2018 and onwards
BO_crime <- subset(BO_crime, BO_crime['YEAR'] == '2018' | BO_crime['YEAR'] == '2019')
#Get rid of NA values for lat and long
BO_crime <-subset(BO_crime, is.na(BO_crime$Long)==FALSE)
BO_crime <- BO_crime[, c('Lat', 'Long', 'INCIDENT_NUMBER')]
#Convert crime csv to geospatial object
BO_crime <- st_as_sf(BO_crime, coords = c("Lat", "Long"), 
                     crs = 4326, agr = "constant")
#Write and read BO_crime shapefile
st_write(BO_crime,'Data/Boston/Crime/Crime_Data_from_2018_to_Present.shp')
BO_crime <- st_read('Data/Boston/Crime/Crime_Data_from_2018_to_Present.shp')
#Trashcans
#Create columns "Lat" and "Long" from "Location"
BO_trashcans <- BO_trashcans %>% separate('Location', into = paste0('Location', 1:2 ), sep = ',')
BO_trashcans$Long <- stringr::str_replace(BO_trashcans$Location1, '\\(', '')
BO_trashcans$Lat <- stringr::str_replace(BO_trashcans$Location2, '\\)', '')
#Convert Long and Lat to numeric
BO_trashcans$Long<- as.numeric(BO_trashcans$Long)
BO_trashcans$Lat<- as.numeric(BO_trashcans$Lat)
#Remove unneeded columns
BO_trashcans <- select(BO_trashcans, 'description', 'Lat', 'Long')
#Get rid of NA values for location
BO_trashcans <- subset(BO_trashcans, is.na(BO_trashcans$Lat)==FALSE)
#Convert crime csv to geospatial object
BO_trashcans <- st_as_sf(BO_trashcans, coords = c("Lat", "Long"), 
                     crs = 4326, agr = "constant")
#Check structure of BO_trashcans
str(BO_trashcans)
#Write and read BO_trashcans shapefile
st_write(BO_trashcans,'Data/Boston/TrashCans/TrashCans.shp')
BO_trashcans <- st_read('Data/Boston/TrashCans/TrashCans.shp')

In [ ]:
#Check coordinate systems
st_crs(BO_streetlights)
st_crs(BO_trashcans)
st_crs(BO_trees)
st_crs(BO_prcnts)
st_crs(BO_crime)

#Reproject to common coordinate systems
#Reproject streetlights to common coordinate system
BO_streetlights <- st_transform(BO_streetlights, "+proj=longlat +datum=WGS84 +no_defs")

#Check that all spatial layers are in the same coordinate system
st_crs(BO_streetlights)
st_crs(BO_trashcans)
st_crs(BO_trees)
st_crs(BO_prcnts)
st_crs(BO_crime)

In [ ]:
#Now let's clean up our layers to include only columns of interest
#Crime
#Add crime count column
BO_crime$crimecount <- 1
#Precincts layer
BO_prcnts <- select(BO_prcnts, 'WARD_PRECI', 'geometry')
#Trashcans
#Add trash count column
BO_trashcans$trashcount <- 1
#Streetlights
BO_streetlights <- select(BO_streetlights, 'objectid', 'geometry')
BO_streetlights$lightcount <- 1
#Trees
BO_trees <- select(BO_trees, 'OBJECTID', 'geometry')
BO_trees$treecount <- 1

In [ ]:
#Let's write a function that takes in all the variables, perform spatial joins on points inside voting precinct, 
#and summarizes by voting precinct 
BO_prcnts_varsfunc <- function(geom, var1, var2, var3, var4) {
  #var1=trees; var2=crime; var3=streetlights; var4=trashcans
  
  #Join first variable
  geom_var1 <- st_join(geom, left = TRUE, var1[length(var1)])
  geom_var1 <- group_by(geom_var1, WARD_PRECI)
  geom_var1 <- summarise(geom_var1, treecount = sum(treecount))
  #Join second variable
  geom_var12 <- st_join(geom_var1, left = TRUE, var2[length(var2)])
  geom_var12 <- group_by(geom_var12, WARD_PRECI, treecount)
  geom_var12 <- summarise(geom_var12, crimecount = sum(crimecount))
  #Join third variable
  geom_var123 <- st_join(geom_var12, left = TRUE, var3[length(var3)])
  geom_var123 <- group_by(geom_var123, WARD_PRECI, treecount, crimecount)
  geom_var123 <- summarise(geom_var123, lightcount = sum(lightcount))
  #Join fourth variable
  geom_var1234 <- st_join(geom_var123, left = TRUE, var4[length(var4)])
  geom_var1234 <- group_by(geom_var1234, WARD_PRECI, treecount, crimecount, lightcount)
  geom_var1234 <- summarise(geom_var1234, trashcount = sum(trashcount))
  
  #Replace NA values with 0 (meaning that the count for a given var is 0 if there was no point within a polygon)
  geom_var1234[is.na(geom_var1234)] <- 0
  geom_vars <- geom_var1234
  return(geom_vars)
}

In [ ]:
#Running our spatial join function for Washington DC
BO_prcnts_vars <- BO_prcnts_varsfunc(BO_prcnts, BO_trees, BO_crime, BO_streetlights, BO_trashcans)
#Save file
#st_write(BO_prcnts_vars, "Data/Boston/ProcessedData/BO_prcnts_vars.shp")

In [ ]:
#Running our spatial join function for Washington DC
BO_prcnts_vars <- BO_prcnts_varsfunc(BO_prcnts, BO_trees, BO_crime, BO_streetlights, BO_trashcans)
#Save file
st_write(BO_prcnts_vars, "Data/Boston/ProcessedData/BO_prcnts_vars.shp")

#First we calculate the area of each precinct in km2
BO_prcnts_vars$km2 <- as.numeric((st_area(BO_prcnts_vars)/1000000))
#Next we divide each variable by the tract area
BO_prcnts_vars$crimecount <- BO_prcnts_vars$crimecount/BO_prcnts_vars$km2
BO_prcnts_vars$lightcount <- BO_prcnts_vars$lightcount/BO_prcnts_vars$km2
BO_prcnts_vars$trashcount <- BO_prcnts_vars$trashcount/BO_prcnts_vars$km2
BO_prcnts_vars$treecount <- BO_prcnts_vars$treecount/BO_prcnts_vars$km2
#Reorder dataframe
BO_prcnts_vars <- BO_prcnts_vars[,c('treecount','crimecount','lightcount','trashcount', 'WARD_PRECI', 'geometry', 'km2')]
#Save file
st_write(BO_prcnts_vars, "Data/Boston/ProcessedData/Boston_prcnts_vars_norm.shp")

### 1.5 Merging all the cities into one dataset

In [ ]:
SF_prcnts_vars <- st_read("Data/SF/ProcessedData/SF_prcnts_vars_norm.shp")
LA_prcnts_vars <- st_read("Data/LA/ProcessedData/LA_prcnts_vars_norm.shp")
PH_prcnts_vars <- st_read("Data/Philadelphia/ProcessedData/PH_prcnts_vars_norm.shp")
DC_prcnts_vars <- st_read("Data/DC/ProcessedData/DC_prcnts_vars_norm.shp")
BO_prcnts_vars <- st_read("Data/Boston/ProcessedData/Boston_prcnts_vars_norm.shp")

In [425]:
#add city name column to each city
#SF_prcnts_vars$CITY <- "San Francisco"
#LA_prcnts_vars$CITY <- "Los Angeles"
#PH_prcnts_vars$CITY <- "Philadelphia"
#DC_prcnts_vars$CITY <- "Washington DC"

#select only common columns
BO_prcnts_vars <- BO_prcnts_vars[, c('treecount','crimecount','lightcount','trashcount', 'geometry','km2')]
SF_prcnts_vars <- SF_prcnts_vars[, c('treecount','crimecount','lightcount','trashcount','CITY', 'geometry','km2')]
LA_prcnts_vars <- LA_prcnts_vars[, c('treecount','crimecount','lightcount','trashcount','CITY', 'geometry','km2')]
PH_prcnts_vars <- PH_prcnts_vars[, c('treecount','crimecount','lightcount','trashcount','CITY', 'geometry','km2')]
DC_prcnts_vars <- DC_prcnts_vars[, c('treecount','crimecount','lightcount','trashcount','CITY', 'geometry','km2')]

In [136]:
#bind all cities row wise
SF_LA_PH_DC_prcnts_vars <- rbind(SF_prcnts_vars, LA_prcnts_vars, PH_prcnts_vars, DC_prcnts_vars)
#bind all cities except philadelphia row wise
SF_LA_DC_prcnts_vars <- rbind(SF_prcnts_vars, LA_prcnts_vars, DC_prcnts_vars)

SF_LA_PH_DC_prcnts_vars$CITY <- as.factor(SF_LA_PH_DC_prcnts_vars$CITY)
st_write(SF_LA_PH_DC_prcnts_vars, 'Data/AllCities/Spatial/SF_LA_PH_DC_prcnts_vars.shp')

Writing layer `SF_LA_PH_DC_prcnts_vars' to data source `Data/AllCities/Spatial/SF_LA_PH_DC_prcnts_vars.shp' using driver `ESRI Shapefile'
Writing 4077 features with 6 fields and geometry type Unknown (any).


In [427]:
#convert to dataframes and write tables
st_geometry(BO_prcnts_vars) <- NULL
st_geometry(SF_LA_PH_DC_prcnts_vars) <- NULL
#all
write.csv(SF_LA_PH_DC_prcnts_vars, file = 'Data/AllCities/Tables/SF_LA_PH_DC_prcnts_vars.csv')
#SF
write.csv(SF_prcnts_vars, file = 'Data/AllCities/Tables/SF_prcnts_vars.csv')
#LA
write.csv(LA_prcnts_vars, file = 'Data/AllCities/Tables/LA_prcnts_vars.csv')
#PH
write.csv(PH_prcnts_vars, file = 'Data/AllCities/Tables/PH_prcnts_vars.csv')
#DC
write.csv(DC_prcnts_vars, file = 'Data/AllCities/Tables/DC_prcnts_vars.csv')
#Boston
write.csv(BO_prcnts_vars, file = 'Data/AllCities/Tables/BO_prcnts_vars.csv')

In [7]:
#row bind data and write final file
SF_LA_PH_DC_BO_prcnts_vars <- rbind(SF_prcnts_vars, LA_prcnts_vars, 
                                           PH_prcnts_vars, DC_prcnts_vars, BO_prcnts_vars)
#create 5 dummy columns (1 and 0 for cities, same for precinct) to capture fixed effects of city and precinct
#create dummy columns for each city
SF_LA_PH_DC_BO_prcnts_vars <- fastDummies::dummy_cols(SF_LA_PH_DC_BO_prcnts_vars, select_columns = "CITY")
#SF_LA_PH_DC_BO_prcnts_vars <- subset(SF_LA_PH_DC_BO_prcnts_vars_clnorm, SF_LA_PH_DC_BO_prcnts_vars_clnorm$crimecount < 1)
write.csv(SF_LA_PH_DC_BO_prcnts_vars, file = 'Data/AllCities/Tables/SF_LA_PH_DC_BO_prcnts_vars.csv')

### Map of US crime by county

In [14]:
US_counties <- st_read('data/County/tl_2016_us_county.shp')
county_crime <- read.csv('data/County/crime_data_w_population_and_crime_rate.csv')

Reading layer `tl_2016_us_county' from data source `C:\Users\Leonardo\Documents\TU_Delft\EPA_Year1\EPA1315\FinalProject\Data\County\tl_2016_us_county.shp' using driver `ESRI Shapefile'
Simple feature collection with 3233 features and 17 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -179.2311 ymin: -14.60181 xmax: 179.8597 ymax: 71.44106
epsg (SRID):    4269
proj4string:    +proj=longlat +datum=NAD83 +no_defs


In [15]:
#separate county name and state
county_crime <- county_crime %>% separate('county_name', into = paste0('county_name', 1:2), sep = ',')
UScounty_crimes <- merge(US_counties, county_crime, by.x = 'NAMELSAD', by.y = 'county_name1')
UScounty_crimes <- UScounty_crimes[, c('crime_rate_per_100000', 'NAME')]
UScounty_crimes <- st_transform(UScounty_crimes, "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs")
st_write(UScounty_crimes, 'Data/County/UScounty_crimes.shp')

Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"

Writing layer `UScounty_crimes' to data source `Data/County/UScounty_crimes.shp' using driver `ESRI Shapefile'
Writing 14525 features with 2 fields and geometry type Multi Polygon.
